In [505]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def encode_categorical_data(df, target_column='Y'):
    df_encoded = df.copy()
    le = LabelEncoder()
    categorical_cols = df_encoded.select_dtypes(include=['object', 'category']).columns
    categorical_cols = [col for col in categorical_cols if col != target_column]
    
    for column in categorical_cols:
        if df_encoded[column].dtype == 'object' or df_encoded[column].dtype.name == 'category':
            df_encoded[column] = le.fit_transform(df_encoded[column].astype(str))
    
    return df_encoded

In [506]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler

def robust_transform(X):
    scaler = RobustScaler()
    if isinstance(X, pd.DataFrame):
        return pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    else:
        X = np.asarray(X)
        return scaler.fit_transform(X)

def apply_robust_transform(X, y):
    X_transformed = robust_transform(X)
    return X_transformed, y

In [507]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

def apply_synthetic_data_to_training(X, y, test_size=0.2, random_state=42):
    # Ensure X is a DataFrame and reset its index
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)
    X = X.reset_index(drop=True)
    
    # Ensure y is a Series and reset its index
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    y = y.reset_index(drop=True)
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Combine X_train and y_train into a single DataFrame
    train_data = pd.concat([X_train, y_train.rename('Y')], axis=1)
    
    # Generate synthetic data using Decision Tree
    synthetic_data = generate_data_decision_tree(train_data)
    
    # Combine the synthetic data with the original training data
    augmented_train_data = pd.concat([train_data, synthetic_data], ignore_index=True)
    
    # Separate features and target for the augmented training data
    X_train_augmented = augmented_train_data.drop('Y', axis=1)
    y_train_augmented = augmented_train_data['Y']
    
    # Combine the augmented training data with the original test data
    X_combined = pd.concat([X_train_augmented, X_test], ignore_index=True)
    y_combined = pd.concat([y_train_augmented, y_test], ignore_index=True)
    
    return X_combined, y_combined

def generate_data_decision_tree(train_data):
    """Generate synthetic data using Decision Tree."""
    X = train_data.drop('Y', axis=1)
    y = train_data['Y']
    
    dt = DecisionTreeRegressor(random_state=42)
    dt.fit(X, y)
    
    synthetic_X = pd.DataFrame(columns=X.columns)
    synthetic_y = []
    
    for _ in range(len(train_data)):
        sample = X.sample(n=1, replace=True)
        synthetic_sample = sample.copy()
        
        for feature in X.columns:
            if np.random.rand() < 0.5:  # 50% chance to modify each feature
                feature_min = X[feature].min()
                feature_max = X[feature].max()
                synthetic_sample[feature] = np.random.uniform(feature_min, feature_max)
        
        synthetic_X = pd.concat([synthetic_X, synthetic_sample], ignore_index=True)
        synthetic_y.append(dt.predict(synthetic_sample)[0])
    
    synthetic_y = pd.Series(synthetic_y, name='Y')
    
    synthetic_data = pd.concat([synthetic_X, synthetic_y], axis=1)
    return synthetic_data


In [508]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

def model_comparison(df, target_column, test_size=0.2, random_state=42, cv=5):
    # Prepare the data
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Dictionary of models and their reduced hyperparameter grids
    models = {
        'Linear Regression': (LinearRegression(), {}),
        'Ridge': (Ridge(), {
            'alpha': [0.1, 1.0, 10.0],
            'solver': ['auto', 'svd', 'cholesky']
        }),
        'Lasso': (Lasso(), {
            'alpha': [0.1, 1.0, 10.0],
            'selection': ['cyclic', 'random']
        }),
        'KNN': (KNeighborsRegressor(), {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance'],
            'p': [1, 2]
        }),
        'Decision Tree': (DecisionTreeRegressor(), {
            'max_depth': [None, 10, 20, 40],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }),
        'Random Forest': (RandomForestRegressor(), {
            'n_estimators': [100, 200, 400],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [2, 4]
        }),
        'Gradient Boosting': (GradientBoostingRegressor(), {
            'n_estimators': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4]
        }),
        'XGBoost': (XGBRegressor(), {
            'n_estimators': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 4],
            'gamma': [0, 0.1]
        }),
        'LightGBM': (LGBMRegressor(), {
            'n_estimators': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [31, 50, 70]
        }),
        'CatBoost': (CatBoostRegressor(verbose=0), {
            'iterations': [100, 200, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'depth': [4, 6, 8]
        })
    }
    
    # Dictionary to store results
    results = {}

    for name, (model, param_grid) in models.items():
        start_time = time.time()
        
        # Perform GridSearchCV
        grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train)
        
        # Get the best model
        best_model = grid_search.best_estimator_

        # Measure training time for best parameters
        best_param_train_start = time.time()
        best_model.fit(X_train_scaled, y_train)
        best_param_train_time = time.time() - best_param_train_start

        # Measure inference time for best parameters
        inference_start_time = time.time()
        y_pred = best_model.predict(X_test_scaled)
        inference_time = time.time() - inference_start_time
        
        # Calculate total computation time
        computation_time = time.time() - start_time
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Perform cross-validation
        cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')
        cv_rmse = np.sqrt(-cv_scores)

        results[name] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2 Score': r2,
            'CV Mean RMSE': np.mean(cv_rmse),
            'CV Std RMSE': np.std(cv_rmse),
            'Training Time (Best Params)': best_param_train_time,
            'Inference Time (Best Params)': inference_time,
            'Computation Time (Total)': computation_time,
            'Best Parameters': grid_search.best_params_
        }

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results).T

    return result_df


In [509]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor

def mlp_comparison(X, y, result_df, test_size=0.2, random_state=42, cv=3):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define the MLP model and its hyperparameter grid
    mlp = MLPRegressor(max_iter=1000, random_state=random_state)
    param_grid = {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive']
    }

    # Perform GridSearchCV with KFold
    start_time = time.time()
    kf = KFold(n_splits=cv, shuffle=True, random_state=random_state)
    grid_search = GridSearchCV(mlp, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Measure training time for best parameters
    best_param_train_start = time.time()
    best_model.fit(X_train_scaled, y_train)
    training_time = time.time() - best_param_train_start

    # Measure inference time for best parameters
    inference_start_time = time.time()
    y_pred = best_model.predict(X_test_scaled)
    inference_time = time.time() - inference_start_time

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Perform cross-validation
    cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores)

    # Store results in the existing result DataFrame
    result_df.loc['MLP'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': grid_search.best_params_
    }

    return result_df, grid_search.best_params_


In [510]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def dnn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.values.reshape(-1, 1)).to(device)

    # Define the DNN model
    class DNN(nn.Module):
        def __init__(self, input_dim, hidden_dims):
            super(DNN, self).__init__()
            layers = []
            for hidden_dim in hidden_dims:
                layers.append(nn.Linear(input_dim, hidden_dim))
                layers.append(nn.ReLU())
                input_dim = hidden_dim
            layers.append(nn.Linear(input_dim, 1))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x).squeeze()

    # Define the objective function for Optuna
    def objective(trial):
        hidden_dims = [trial.suggest_int(f'hidden_dim_{i}', 32, 256) for i in range(3)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        input_dim = X_train.shape[1]
        model = DNN(input_dim, hidden_dims).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    start_time = time.time()

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params

    input_dim = X_train.shape[1]
    best_model = DNN(input_dim, [best_params[f'hidden_dim_{i}'] for i in range(3)]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train.values[train_index], y_train.values[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = DNN(input_dim, [best_params[f'hidden_dim_{i}'] for i in range(3)]).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    computation_time = time.time() - start_time

    result_df.loc['DNN'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [511]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def dcn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.values.reshape(-1, 1)).to(device)

    class CrossLayer(nn.Module):
        def __init__(self, input_dim):
            super(CrossLayer, self).__init__()
            self.weight = nn.Parameter(torch.Tensor(input_dim, 1))
            self.bias = nn.Parameter(torch.Tensor(input_dim, 1))
            nn.init.xavier_uniform_(self.weight)
            nn.init.zeros_(self.bias)

        def forward(self, x0, x):
            x = x.unsqueeze(2)
            x0 = x0.unsqueeze(2)
            interaction = torch.matmul(x0, torch.matmul(x.transpose(1, 2), self.weight))
            return x0.squeeze(2) + interaction.squeeze(2) + self.bias.T

    class DCN(nn.Module):
        def __init__(self, input_dim, cross_layers, hidden_layers):
            super(DCN, self).__init__()
            self.cross_layers = nn.ModuleList([CrossLayer(input_dim) for _ in range(cross_layers)])
            
            deep_layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                deep_layers.append(nn.ReLU())
            self.deep_net = nn.Sequential(*deep_layers)
            
            self.final_layer = nn.Linear(input_dim + hidden_layers[-1], 1)

        def forward(self, x):
            cross_out = x
            for layer in self.cross_layers:
                cross_out = layer(x, cross_out)
            deep_out = self.deep_net(x)
            concat_out = torch.cat([cross_out, deep_out], dim=1)
            return self.final_layer(concat_out).squeeze()

    def objective(trial):
        cross_layers = trial.suggest_int('cross_layers', 1, 5)
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        input_dim = X_train.shape[1]
        model = DCN(input_dim, cross_layers, hidden_layers).to(device)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    start_time = time.time()

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params

    input_dim = X_train.shape[1]
    best_model = DCN(input_dim, best_params['cross_layers'], 
                     [best_params[f'hidden_layer_{i}'] for i in range(3)]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y.squeeze())
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train.values[train_index], y_train.values[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = DCN(input_dim, best_params['cross_layers'], 
                         [best_params[f'hidden_layer_{i}'] for i in range(3)]).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    computation_time = time.time() - start_time

    result_df.loc['DCN'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [512]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def wide_and_deep_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.values.reshape(-1, 1)).to(device)

    class WideAndDeepNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers):
            super(WideAndDeepNetwork, self).__init__()
            
            # Wide part
            self.wide = nn.Linear(input_dim, 1)
            
            # Deep part
            deep_layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    deep_layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    deep_layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                deep_layers.append(nn.ReLU())
            deep_layers.append(nn.Linear(hidden_layers[-1], 1))
            self.deep = nn.Sequential(*deep_layers)

        def forward(self, x):
            wide_out = self.wide(x)
            deep_out = self.deep(x)
            return wide_out + deep_out

    def objective(trial):
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        input_dim = X_train.shape[1]
        model = WideAndDeepNetwork(input_dim, hidden_layers).to(device)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    start_time = time.time()

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params

    input_dim = X_train.shape[1]
    best_model = WideAndDeepNetwork(input_dim, 
                                    [best_params[f'hidden_layer_{i}'] for i in range(3)]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy().squeeze()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train.values[train_index], y_train.values[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = WideAndDeepNetwork(input_dim, 
                                        [best_params[f'hidden_layer_{i}'] for i in range(3)]).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    computation_time = time.time() - start_time

    result_df.loc['Wide_and_Deep'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [513]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import optuna

def xgb_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    start_time = time.time()

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers):
            super(NeuralNetwork, self).__init__()
            layers = []
            prev_dim = input_dim
            for hidden_dim in hidden_layers:
                layers.append(nn.Linear(prev_dim, hidden_dim))
                layers.append(nn.ReLU())
                prev_dim = hidden_dim
            layers.append(nn.Linear(prev_dim, 1))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x).squeeze()

    def objective(trial):
        # Define hyperparameters to tune for XGBoost
        xgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
        }

        # Train XGBoost model
        xgb_model = XGBRegressor(**xgb_params)
        xgb_model.fit(X_train_scaled, y_train)
        
        # Extract features using XGBoost
        X_train_transformed = xgb_model.apply(X_train_scaled)
        X_test_transformed = xgb_model.apply(X_test_scaled)
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
        y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = []
        for i in range(3):  # Allow up to 3 hidden layers
            if trial.suggest_categorical(f'use_hidden_layer_{i}', [True, False]):
                hidden_layers.append(trial.suggest_int(f'hidden_layer_{i}', 32, 256))
        
        nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        model = NeuralNetwork(input_dim, hidden_layers).to(device)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_transformed_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final XGBoost model with the best hyperparameters
    xgb_best_params = {
        'n_estimators': best_params['n_estimators'],
        'max_depth': best_params['max_depth'],
        'learning_rate': best_params['xgb_learning_rate'],
        'subsample': best_params['subsample'],
        'colsample_bytree': best_params['colsample_bytree']
    }
    xgb_model = XGBRegressor(**xgb_best_params)
    xgb_model.fit(X_train_scaled, y_train)

    # Extract features using XGBoost
    X_train_transformed = xgb_model.apply(X_train_scaled)
    X_test_transformed = xgb_model.apply(X_test_scaled)

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }

    for i in range(3):  # Assuming max 3 hidden layers
        if f'use_hidden_layer_{i}' in best_params and best_params[f'use_hidden_layer_{i}']:
            nn_best_params['hidden_layers'].append(best_params[f'hidden_layer_{i}'])

    input_dim = X_train_transformed.shape[1]
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_transformed_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        xgb_model.fit(X_fold_train, y_fold_train)
        X_fold_train_transformed = xgb_model.apply(X_fold_train)
        X_fold_val_transformed = xgb_model.apply(X_fold_val)

        X_fold_train_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    computation_time = time.time() - start_time

    result_df.loc['XGBoost + NN'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [514]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor
import optuna

def lgbm_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], 1))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x).squeeze()

    def objective(trial):
        # Define hyperparameters to tune for LightGBM
        lgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('lgb_learning_rate', 1e-4, 1e-1, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
        }

        # Train LightGBM model
        lgb_model = LGBMRegressor(**lgb_params)
        lgb_model.fit(X_train_scaled, y_train)
        
        # Extract features using LightGBM
        X_train_transformed = lgb_model.predict(X_train_scaled).reshape(-1, 1)
        X_test_transformed = lgb_model.predict(X_test_scaled).reshape(-1, 1)
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
        y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        nn_learning_rate = trial.suggest_float('nn_learning_rate', 1e-4, 1e-1, log=True)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        model = NeuralNetwork(input_dim, hidden_layers).to(device)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_transformed_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final LightGBM model with the best hyperparameters
    lgb_best_params = {
        'n_estimators': best_params['n_estimators'],
        'max_depth': best_params['max_depth'],
        'learning_rate': best_params['lgb_learning_rate'],
        'num_leaves': best_params['num_leaves'],
        'subsample': best_params['subsample'],
        'colsample_bytree': best_params['colsample_bytree']
    }
    lgb_model = LGBMRegressor(**lgb_best_params)
    lgb_model.fit(X_train_scaled, y_train)

    # Extract features using LightGBM
    X_train_transformed = lgb_model.predict(X_train_scaled).reshape(-1, 1)
    X_test_transformed = lgb_model.predict(X_test_scaled).reshape(-1, 1)

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }
    input_dim = X_train_transformed.shape[1]
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y.squeeze())
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_transformed_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        lgb_model.fit(X_fold_train, y_fold_train)
        X_fold_train_transformed = lgb_model.predict(X_fold_train).reshape(-1, 1)
        X_fold_val_transformed = lgb_model.predict(X_fold_val).reshape(-1, 1)

        X_fold_train_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['LightGBM + NN'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [515]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def autoint_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    class AutoInt(nn.Module):
        def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
            super(AutoInt, self).__init__()
            self.embedding = nn.Linear(input_dim, embedding_dim)
            self.attention_layers = nn.ModuleList([
                nn.MultiheadAttention(embedding_dim, num_heads) for _ in range(num_layers)
            ])
            self.fc = nn.Linear(embedding_dim, 1)

        def forward(self, x):
            x = self.embedding(x).unsqueeze(1)
            for attn_layer in self.attention_layers:
                x, _ = attn_layer(x, x, x)
            x = x.squeeze(1)
            x = self.fc(x)
            return x

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], 1))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x).squeeze()

    def objective(trial):
        # Define hyperparameters to tune for AutoInt
        num_heads = trial.suggest_int('num_heads', 1, 8)
        embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        
        # Train AutoInt model
        autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        
        for epoch in range(10):  # Fixed number of epochs for AutoInt
            autoint_model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = autoint_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
        
        # Extract features using AutoInt
        autoint_model.eval()
        with torch.no_grad():
            X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
            X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        model = NeuralNetwork(input_dim, hidden_layers).to(device)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_transformed_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final AutoInt model with the best hyperparameters
    embedding_dim = best_params['embedding_dim']
    num_heads = best_params['num_heads']
    num_layers = best_params['num_layers']
    autoint_model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(autoint_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for epoch in range(10):  # Fixed number of epochs for AutoInt
        autoint_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = autoint_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Extract features using AutoInt
    autoint_model.eval()
    with torch.no_grad():
        X_train_transformed = autoint_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = autoint_model.embedding(X_test_tensor).cpu().numpy()

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }
    input_dim = X_train_transformed.shape[1]
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y.squeeze())
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_transformed_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_autoint_model = AutoInt(X_fold_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        fold_optimizer = optim.Adam(fold_autoint_model.parameters(), lr=0.001)
        fold_criterion = nn.MSELoss()
        fold_train_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=32, shuffle=True)

        for epoch in range(10):  # Fixed number of epochs for AutoInt
            fold_autoint_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_autoint_model(batch_X)
                loss = fold_criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_autoint_model.eval()
        with torch.no_grad():
            X_fold_train_transformed = fold_autoint_model.embedding(X_fold_train_tensor).cpu().numpy()
            X_fold_val_transformed = fold_autoint_model.embedding(X_fold_val_tensor).cpu().numpy()

        X_fold_train_transformed_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        X_fold_val_transformed_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_train_dataset = TensorDataset(X_fold_train_transformed_tensor, y_fold_train_tensor)
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_transformed_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    # Calculate total computation time
    computation_time = time.time() - start_time

       # Store results in the existing result DataFrame
    result_df.loc['AutoInt + NN'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params

In [516]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def ft_transformer_nn_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    class FTTransformer(nn.Module):
        def __init__(self, input_dim, embedding_dim, num_heads, num_layers):
            super(FTTransformer, self).__init__()
            self.embedding = nn.Linear(input_dim, embedding_dim)
            self.transformer_layers = nn.ModuleList([
                nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads) for _ in range(num_layers)
            ])
            self.fc = nn.Linear(embedding_dim, input_dim)

        def forward(self, x):
            x = self.embedding(x).unsqueeze(1)
            for transformer_layer in self.transformer_layers:
                x = transformer_layer(x)
            x = x.squeeze(1)
            x = self.fc(x)
            return x

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], 1))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x).squeeze()

    def objective(trial):
        # Define hyperparameters to tune for FT-Transformer
        num_heads = trial.suggest_int('num_heads', 1, 8)
        embedding_dim = trial.suggest_int('embedding_dim', num_heads, 64, step=num_heads)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        
        # Train FT-Transformer model
        ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        
        for epoch in range(10):  # Fixed number of epochs for FT-Transformer
            ft_transformer_model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = ft_transformer_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
        
        # Extract features using FT-Transformer
        ft_transformer_model.eval()
        with torch.no_grad():
            X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
            X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()
        
        # Convert to PyTorch tensors
        X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
        X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)
        
        # Define hyperparameters to tune for Neural Network
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(3)]
        nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train_transformed.shape[1]
        model = NeuralNetwork(input_dim, hidden_layers).to(device)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=nn_learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_transformed_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final FT-Transformer model with the best hyperparameters
    embedding_dim = best_params['embedding_dim']
    num_heads = best_params['num_heads']
    num_layers = best_params['num_layers']
    ft_transformer_model = FTTransformer(X_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
    optimizer = optim.Adam(ft_transformer_model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    for epoch in range(10):  # Fixed number of epochs for FT-Transformer
        ft_transformer_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = ft_transformer_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Extract features using FT-Transformer
    ft_transformer_model.eval()
    with torch.no_grad():
        X_train_transformed = ft_transformer_model.embedding(X_train_tensor).cpu().numpy()
        X_test_transformed = ft_transformer_model.embedding(X_test_tensor).cpu().numpy()

    # Convert to PyTorch tensors
    X_train_transformed_tensor = torch.FloatTensor(X_train_transformed).to(device)
    X_test_transformed_tensor = torch.FloatTensor(X_test_transformed).to(device)

    # Train the final Neural Network model with the best hyperparameters
    nn_best_params = {
        'hidden_layers': [best_params[f'hidden_layer_{i}'] for i in range(3)],
        'learning_rate': best_params['nn_learning_rate'],
        'batch_size': best_params['batch_size'],
        'num_epochs': best_params['num_epochs']
    }
    input_dim = X_train_transformed.shape[1]
    best_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=nn_best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_transformed_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(nn_best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_transformed_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_ft_transformer_model = FTTransformer(X_fold_train.shape[1], embedding_dim, num_heads, num_layers).to(device)
        fold_optimizer = optim.Adam(fold_ft_transformer_model.parameters(), lr=0.001)
        fold_criterion = nn.MSELoss()

        fold_train_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=32, shuffle=True)

        for epoch in range(10):
            fold_ft_transformer_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_ft_transformer_model(batch_X)
                loss = fold_criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_ft_transformer_model.eval()
        with torch.no_grad():
            X_fold_train_transformed = fold_ft_transformer_model.embedding(X_fold_train_tensor).cpu().numpy()
            X_fold_val_transformed = fold_ft_transformer_model.embedding(X_fold_val_tensor).cpu().numpy()

        X_fold_train_transformed_tensor = torch.FloatTensor(X_fold_train_transformed).to(device)
        X_fold_val_transformed_tensor = torch.FloatTensor(X_fold_val_transformed).to(device)

        fold_model = NeuralNetwork(input_dim, nn_best_params['hidden_layers']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=nn_best_params['learning_rate'])
        fold_train_dataset = TensorDataset(X_fold_train_transformed_tensor, y_fold_train_tensor)
        fold_train_loader = DataLoader(fold_train_dataset, batch_size=nn_best_params['batch_size'], shuffle=True)

        for epoch in range(nn_best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_train_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_transformed_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    cv_mean_rmse = np.mean(cv_rmse)
    cv_std_rmse = np.std(cv_rmse)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['FT-Transformer'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': cv_mean_rmse,
        'CV Std RMSE': cv_std_rmse,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [517]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def neural_architecture_search(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    class NeuralNetwork(nn.Module):
        def __init__(self, input_dim, hidden_layers):
            super(NeuralNetwork, self).__init__()
            layers = []
            for i in range(len(hidden_layers)):
                if i == 0:
                    layers.append(nn.Linear(input_dim, hidden_layers[i]))
                else:
                    layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_layers[-1], 1))
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x).squeeze()

    def objective(trial):
        # Define hyperparameters to tune for Neural Network
        num_layers = trial.suggest_int('num_layers', 1, 5)
        hidden_layers = [trial.suggest_int(f'hidden_layer_{i}', 32, 256) for i in range(num_layers)]
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the Neural Network model
        input_dim = X_train.shape[1]
        model = NeuralNetwork(input_dim, hidden_layers).to(device)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final Neural Network model with the best hyperparameters
    input_dim = X_train.shape[1]
    best_model = NeuralNetwork(input_dim, 
                               [best_params[f'hidden_layer_{i}'] for i in range(best_params['num_layers'])]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = NeuralNetwork(input_dim, 
                                   [best_params[f'hidden_layer_{i}'] for i in range(best_params['num_layers'])]).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    cv_mean_rmse = np.mean(cv_rmse)
    cv_std_rmse = np.std(cv_rmse)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['Neural Architecture Search'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': cv_mean_rmse,
        'CV Std RMSE': cv_std_rmse,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [518]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def kan_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    class KAN(nn.Module):
        def __init__(self, input_dim, hidden_dim):
            super(KAN, self).__init__()
            self.hidden_layer = nn.Linear(input_dim, hidden_dim)
            self.activation = nn.ReLU()
            self.output_layer = nn.Linear(hidden_dim, 1)

        def forward(self, x):
            h = self.activation(self.hidden_layer(x))
            out = self.output_layer(h)
            return out.squeeze()

    def objective(trial):
        hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        input_dim = X_train.shape[1]
        model = KAN(input_dim, hidden_dim).to(device)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params

    input_dim = X_train.shape[1]
    best_model = KAN(input_dim, best_params['hidden_dim']).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y.squeeze())
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        y_true = y_test_tensor.cpu().numpy().squeeze()
        y_pred = predictions.cpu().numpy()

        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = KAN(input_dim, best_params['hidden_dim']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    computation_time = time.time() - start_time

    result_df.loc['KAN'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [519]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def node_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=10, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    class NODE(nn.Module):
        def __init__(self, input_dim, num_layers, num_trees, tree_dim):
            super(NODE, self).__init__()
            self.layers = nn.ModuleList()
            for _ in range(num_layers):
                layer = nn.ModuleList()
                for _ in range(num_trees):
                    tree = nn.Sequential(
                        nn.Linear(input_dim, tree_dim),
                        nn.ReLU(),
                        nn.Linear(tree_dim, 1)
                    )
                    layer.append(tree)
                self.layers.append(layer)
            self.output = nn.Linear(num_layers * num_trees, 1)

        def forward(self, x):
            tree_outputs = []
            for layer in self.layers:
                layer_outputs = []
                for tree in layer:
                    layer_outputs.append(tree(x))
                layer_output = torch.cat(layer_outputs, dim=1)
                tree_outputs.append(layer_output)
            x = torch.cat(tree_outputs, dim=1)
            return self.output(x).squeeze()

    def objective(trial):
        # Define hyperparameters to tune for NODE
        num_layers = trial.suggest_int('num_layers', 1, 5)
        num_trees = trial.suggest_int('num_trees', 1, 10)
        tree_dim = trial.suggest_int('tree_dim', 8, 64)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        num_epochs = trial.suggest_int('num_epochs', 10, 100)

        # Create the NODE model
        input_dim = X_train.shape[1]
        model = NODE(input_dim, num_layers, num_trees, tree_dim).to(device)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Create DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Training
        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final NODE model with the best hyperparameters
    input_dim = X_train.shape[1]
    best_model = NODE(input_dim, 
                      best_params['num_layers'], 
                      best_params['num_trees'], 
                      best_params['tree_dim']).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y.squeeze())
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    # Evaluation
    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        y_pred = predictions.cpu().numpy()
        y_true = y_test_tensor.cpu().numpy().squeeze()
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = NODE(input_dim, 
                          best_params['num_layers'], 
                          best_params['num_trees'], 
                          best_params['tree_dim']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    computation_time = time.time() - start_time

    result_df.loc['NODE'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': np.mean(cv_rmse),
        'CV Std RMSE': np.std(cv_rmse),
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [520]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pytorch_tabnet.tab_model import TabNetRegressor
import optuna

def tabnet_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=5, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Reshape y to be 2D
    y = y.reshape(-1, 1)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    def objective(trial):
        # Define hyperparameters to tune for TabNet
        n_d = trial.suggest_int('n_d', 8, 64)
        n_a = trial.suggest_int('n_a', 8, 64)
        n_steps = trial.suggest_int('n_steps', 3, 10)
        gamma = trial.suggest_float('gamma', 1.0, 2.0)
        lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-6, 1e-3)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128,256])
        num_epochs = trial.suggest_int('num_epochs', 10, 40)

        # Create the TabNet model
        model = TabNetRegressor(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=learning_rate),
            device_name=device
        )

        # Training
        model.fit(
            X_train=X_train_scaled, y_train=y_train,
            eval_set=[(X_test_scaled, y_test)],
            eval_name=['val'],
            eval_metric=['rmse'],
            max_epochs=num_epochs,
            patience=10,
            batch_size=batch_size,
            virtual_batch_size=batch_size // 2,
            num_workers=0,
            drop_last=False
        )

        # Evaluation
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)

        return mse

    # Perform hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Train the final TabNet model with the best hyperparameters
    best_model = TabNetRegressor(
        n_d=best_params['n_d'],
        n_a=best_params['n_a'],
        n_steps=best_params['n_steps'],
        gamma=best_params['gamma'],
        lambda_sparse=best_params['lambda_sparse'],
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=best_params['learning_rate']),
        device_name=device
    )

    training_start_time = time.time()
    best_model.fit(
        X_train=X_train_scaled, y_train=y_train,
        eval_set=[(X_test_scaled, y_test)],
        eval_name=['val'],
        eval_metric=['rmse'],
        max_epochs=best_params['num_epochs'],
        patience=10,
        batch_size=best_params['batch_size'],
        virtual_batch_size=best_params['batch_size'] // 2,
        num_workers=0,
        drop_last=False
    )
    training_time = time.time() - training_start_time

    # Evaluation
    inference_start_time = time.time()
    y_pred = best_model.predict(X_test_scaled)
    inference_time = time.time() - inference_start_time

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Cross-validation
    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        fold_model = TabNetRegressor(
            n_d=best_params['n_d'],
            n_a=best_params['n_a'],
            n_steps=best_params['n_steps'],
            gamma=best_params['gamma'],
            lambda_sparse=best_params['lambda_sparse'],
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=best_params['learning_rate']),
            device_name=device
        )

        fold_model.fit(
            X_train=X_fold_train, y_train=y_fold_train,
            eval_set=[(X_fold_val, y_fold_val)],
            eval_name=['val'],
            eval_metric=['rmse'],
            max_epochs=best_params['num_epochs'],
            patience=10,
            batch_size=best_params['batch_size'],
            virtual_batch_size=best_params['batch_size'] // 2,
            num_workers=0,
            drop_last=False
        )

        y_fold_pred = fold_model.predict(X_fold_val)
        fold_rmse = np.sqrt(mean_squared_error(y_fold_val, y_fold_pred))
        cv_rmse.append(fold_rmse)

    cv_mean_rmse = np.mean(cv_rmse)
    cv_std_rmse = np.std(cv_rmse)

    # Calculate total computation time
    computation_time = time.time() - start_time

    # Store results in the existing result DataFrame
    result_df.loc['TabNet'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': cv_mean_rmse,
        'CV Std RMSE': cv_std_rmse,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [521]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

def saint_comparison(X, y, result_df, test_size=0.2, random_state=42, n_trials=5, n_folds=5):
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Ensure X and y are numpy arrays
    X = X.values if isinstance(X, pd.DataFrame) else X
    y = y.values if isinstance(y, pd.Series) else y

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1)).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1)).to(device)

    class SAINT(nn.Module):
        def __init__(self, input_dim, dim, depth, heads, mlp_dim, dropout=0.1):
            super(SAINT, self).__init__()
            self.embeds = nn.Linear(input_dim, dim)
            self.transformer = nn.TransformerEncoder(
                nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout),
                num_layers=depth
            )
            self.mlp_head = nn.Sequential(
                nn.LayerNorm(dim),
                nn.Linear(dim, 1)
            )

        def forward(self, x):
            x = self.embeds(x)
            x = x.unsqueeze(1)  # Add sequence dimension
            x = self.transformer(x)
            x = x.squeeze(1)  # Remove sequence dimension
            return self.mlp_head(x).squeeze()

    def objective(trial):
        heads = trial.suggest_int('heads', 1, 8)
        dim = trial.suggest_int('dim', heads, 256, step=heads)
        depth = trial.suggest_int('depth', 1, 6)
        mlp_dim = trial.suggest_int('mlp_dim', 32, 256)
        dropout = trial.suggest_float('dropout', 0.0, 0.5)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128,256])
        num_epochs = trial.suggest_int('num_epochs', 10, 40)

        input_dim = X_train.shape[1]
        model = SAINT(input_dim, dim, depth, heads, mlp_dim, dropout).to(device)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            mse = mean_squared_error(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())

        return mse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params

    input_dim = X_train.shape[1]
    best_model = SAINT(input_dim, best_params['dim'], best_params['depth'], 
                       best_params['heads'], best_params['mlp_dim'], 
                       best_params['dropout']).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

    training_start_time = time.time()
    for epoch in range(best_params['num_epochs']):
        best_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = best_model(batch_X)
            loss = criterion(outputs, batch_y.squeeze())
            loss.backward()
            optimizer.step()

    training_time = time.time() - training_start_time

    best_model.eval()
    with torch.no_grad():
        inference_start_time = time.time()
        predictions = best_model(X_test_tensor)
        inference_time = time.time() - inference_start_time

        y_true = y_test_tensor.cpu().numpy().squeeze()
        y_pred = predictions.cpu().numpy()

        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    cv_rmse = []
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        X_fold_train, X_fold_val = X_train_scaled[train_index], X_train_scaled[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]

        X_fold_train_tensor = torch.FloatTensor(X_fold_train).to(device)
        y_fold_train_tensor = torch.FloatTensor(y_fold_train.reshape(-1, 1)).to(device)
        X_fold_val_tensor = torch.FloatTensor(X_fold_val).to(device)
        y_fold_val_tensor = torch.FloatTensor(y_fold_val.reshape(-1, 1)).to(device)

        fold_model = SAINT(input_dim, best_params['dim'], best_params['depth'], 
                           best_params['heads'], best_params['mlp_dim'], 
                           best_params['dropout']).to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=best_params['learning_rate'])
        fold_dataset = TensorDataset(X_fold_train_tensor, y_fold_train_tensor)
        fold_loader = DataLoader(fold_dataset, batch_size=best_params['batch_size'], shuffle=True)

        for epoch in range(best_params['num_epochs']):
            fold_model.train()
            for batch_X, batch_y in fold_loader:
                fold_optimizer.zero_grad()
                outputs = fold_model(batch_X)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                fold_optimizer.step()

        fold_model.eval()
        with torch.no_grad():
            fold_predictions = fold_model(X_fold_val_tensor)
            fold_mse = mean_squared_error(y_fold_val_tensor.cpu().numpy(), fold_predictions.cpu().numpy())
            cv_rmse.append(np.sqrt(fold_mse))

    cv_mean_rmse = np.mean(cv_rmse)
    cv_std_rmse = np.std(cv_rmse)

    computation_time = time.time() - start_time

    result_df.loc['SAINT'] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2 Score': r2,
        'CV Mean RMSE': cv_mean_rmse,
        'CV Std RMSE': cv_std_rmse,
        'Training Time (Best Params)': training_time,
        'Inference Time (Best Params)': inference_time,
        'Computation Time (Total)': computation_time,
        'Best Parameters': best_params
    }

    return result_df, best_params


In [522]:
file_prefix = "crop"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
df = encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_robust_transform(X, y)
#X, y = apply_synthetic_data_to_training(X, y)
'''
result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
'''
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/regression/{file_prefix}_result.csv', index=True)

[I 2024-08-06 10:41:11,568] A new study created in memory with name: no-name-3a6281e6-829b-47bc-8554-cd5dcd1da1fa


Using device: cpu


c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 256] and step=5, but the range is not divisible by `step`. It will be replaced by [5, 255].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_4912\301764479.py:62: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-06 10:41:26,496] Trial 0 f

                                       MSE         RMSE          MAE  \
Linear Regression           4889229.831828  2211.160291  1691.734253   
Ridge                        4883088.90566  2209.771234  1690.705258   
Lasso                       4865416.912044  2205.769007  1688.270692   
KNN                          3273818.03677  1809.369514  1027.592047   
Decision Tree               8273080.036686  2876.296236  1650.821209   
Random Forest               3824053.360395  1955.518693  1397.043044   
Gradient Boosting           4477412.204499  2115.989651   1623.35943   
XGBoost                     4467102.980339  2113.552219  1630.558773   
LightGBM                     3659382.94254   1912.95137   1404.21618   
CatBoost                    4676739.350386  2162.577016  1655.313719   
MLP                         4847092.507387  2201.611343  1687.336878   
DNN                              5001051.5  2236.302979  1701.878906   
DCN                              4962697.5  2227.711182  1694.13

In [523]:
file_prefix = "bike"  # Change this to any word you like
df =  pd.read_csv(f'Dataset/{file_prefix}.csv')
df = encode_categorical_data(df)
X = df.drop('Y', axis=1)
y = df['Y']
X, y = apply_robust_transform(X, y)
#X, y = apply_synthetic_data_to_training(X, y)

result = model_comparison(df, 'Y')
print(result)
result, best_params = mlp_comparison(X, y, result)
print(result)
result, best_params = dnn_comparison(X, y, result)
print(result)
result, best_params = dcn_comparison(X, y, result)
print(result)
result, best_params = wide_and_deep_comparison(X, y, result)
print(result)
result, best_params = xgb_nn_comparison(X, y, result)
print(result)
result, best_params = lgbm_nn_comparison(X, y, result)
print(result)
result, best_params = autoint_nn_comparison(X, y, result)
print(result)
result, best_params = ft_transformer_nn_comparison(X, y, result)
print(result)
result, best_params = neural_architecture_search(X, y, result)
print(result)
result, best_params = kan_comparison(X, y, result)
print(result)
result, best_params = node_comparison(X, y, result)
print(result)
result, best_params = tabnet_comparison(X, y, result)
print(result)
result, best_params = saint_comparison(X, y, result)
print(result)

result.to_csv(f'result/comparison/regression/{file_prefix}_result.csv', index=True)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000232 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1595
[LightGBM] [Info] Number of data points in the train set: 584, number of used features: 14
[LightGBM] [Info] Start training from score 4546.361301
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
60 fits failed out of a total of 288.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 751, in fit

                             MSE        RMSE         MAE  R2 Score  \
Linear Regression            0.0         0.0         0.0       1.0   
Ridge                   0.242336    0.492277    0.398738       1.0   
Lasso                   0.013676    0.116946    0.096502       1.0   
KNN                161030.811799  401.286446  296.465121  0.952954   
Decision Tree       30628.274734  175.009356  123.978311  0.991052   
Random Forest       11178.686815  105.729309   65.787632  0.996734   
Gradient Boosting    8478.942997   92.081176     64.2453  0.997523   
XGBoost              9031.190737   95.032577    66.63099  0.997361   
LightGBM            12274.312832  110.789498   72.435986  0.996414   
CatBoost             6256.066985   79.095303   61.673092  0.998172   
MLP                112512.623433  335.429014  264.401499  0.967129   

                  CV Mean RMSE CV Std RMSE Training Time (Best Params)  \
Linear Regression          0.0         0.0                    0.001997   
Ridge      

[I 2024-08-06 10:48:33,429] Trial 0 finished with value: 3675501.5 and parameters: {'hidden_dim_0': 252, 'hidden_dim_1': 241, 'hidden_dim_2': 33, 'learning_rate': 0.03535070946595991, 'batch_size': 256, 'num_epochs': 82}. Best is trial 0 with value: 3675501.5.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_4912\3425235463.py:49: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Tommy\anaconda3\Lib\site-packages\to

                             MSE         RMSE          MAE  R2 Score  \
Linear Regression            0.0          0.0          0.0       1.0   
Ridge                   0.242336     0.492277     0.398738       1.0   
Lasso                   0.013676     0.116946     0.096502       1.0   
KNN                161030.811799   401.286446   296.465121  0.952954   
Decision Tree       30628.274734   175.009356   123.978311  0.991052   
Random Forest       11178.686815   105.729309    65.787632  0.996734   
Gradient Boosting    8478.942997    92.081176      64.2453  0.997523   
XGBoost              9031.190737    95.032577     66.63099  0.997361   
LightGBM            12274.312832   110.789498    72.435986  0.996414   
CatBoost             6256.066985    79.095303    61.673092  0.998172   
MLP                112512.623433   335.429014   264.401499  0.967129   
DNN                    3837166.5  1958.868652  1641.655029 -0.121047   

                  CV Mean RMSE CV Std RMSE Training Time (Best 

...

epoch 0  | loss: 24498725.57265| val_rmse: 4920.85647|  0:00:00s
epoch 1  | loss: 24331322.90598| val_rmse: 4870.4998|  0:00:01s
epoch 2  | loss: 24052455.7265| val_rmse: 4847.83747|  0:00:02s
epoch 3  | loss: 23510567.62393| val_rmse: 4769.44624|  0:00:02s
epoch 4  | loss: 22564583.24786| val_rmse: 4569.64282|  0:00:03s
epoch 5  | loss: 20996712.18803| val_rmse: 4206.66165|  0:00:04s
epoch 6  | loss: 18488050.0| val_rmse: 3727.02999|  0:00:05s
epoch 7  | loss: 15182956.94017| val_rmse: 3004.43316|  0:00:05s
epoch 8  | loss: 11095453.80342| val_rmse: 1734.46439|  0:00:06s
epoch 9  | loss: 6745256.61752| val_rmse: 903.92626|  0:00:07s
epoch 10 | loss: 3025149.5| val_rmse: 2414.38468|  0:00:08s
epoch 11 | loss: 891748.80101| val_rmse: 3096.08453|  0:00:09s
epoch 12 | loss: 618228.76656| val_rmse: 3533.15333|  0:00:09s
epoch 13 | loss: 585663.2703| val_rmse: 3127.70878|  0:00:10s
epoch 14 | loss: 606631.04808| val_rmse: 2297.85575|  0:00:11s
epoch 15 | loss: 479162.47569| val_rmse: 1549.1

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-08-06 11:01:40,724] A new study created in memory with name: no-name-9b091097-93c8-4796-b545-3b00490a06cd


                                      MSE         RMSE          MAE  R2 Score  \
Linear Regression                     0.0          0.0          0.0       1.0   
Ridge                            0.242336     0.492277     0.398738       1.0   
Lasso                            0.013676     0.116946     0.096502       1.0   
KNN                         161030.811799   401.286446   296.465121  0.952954   
Decision Tree                30628.274734   175.009356   123.978311  0.991052   
Random Forest                11178.686815   105.729309    65.787632  0.996734   
Gradient Boosting             8478.942997    92.081176      64.2453  0.997523   
XGBoost                       9031.190737    95.032577     66.63099  0.997361   
LightGBM                     12274.312832   110.789498    72.435986  0.996414   
CatBoost                      6256.066985    79.095303    61.673092  0.998172   
MLP                         112512.623433   335.429014   264.401499  0.967129   
DNN                         

c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 256] and step=5, but the range is not divisible by `step`. It will be replaced by [5, 255].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_4912\301764479.py:62: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-08-06 11:01:57,007] Trial 0 f

                                      MSE         RMSE          MAE  R2 Score  \
Linear Regression                     0.0          0.0          0.0       1.0   
Ridge                            0.242336     0.492277     0.398738       1.0   
Lasso                            0.013676     0.116946     0.096502       1.0   
KNN                         161030.811799   401.286446   296.465121  0.952954   
Decision Tree                30628.274734   175.009356   123.978311  0.991052   
Random Forest                11178.686815   105.729309    65.787632  0.996734   
Gradient Boosting             8478.942997    92.081176      64.2453  0.997523   
XGBoost                       9031.190737    95.032577     66.63099  0.997361   
LightGBM                     12274.312832   110.789498    72.435986  0.996414   
CatBoost                      6256.066985    79.095303    61.673092  0.998172   
MLP                         112512.623433   335.429014   264.401499  0.967129   
DNN                         